In [1]:
import pandas as pd
import numpy as np
from siuba import *
from calitp import *

In [2]:
pd.set_option("display.max_columns", 100)

In [3]:
bc = to_snakecase(
    pd.read_excel(
        "gs://calitp-analytics-data/data-analyses/grants/Grant+Projects_3_17_2022.xlsx"
    )
)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
bc.funding_program.value_counts()

5310 Trad          964
Section 5311       590
CARES Act          166
CRRSAA             153
5310 Exp           151
Section 5311(f)    109
5339 (State)        95
Toll Credits        52
ARPA                50
5339 (National)     48
CARES Act (F)       43
5311(f) Cont        41
CMAQ (FTA 5311)     31
5311(f) Round 2     27
Name: funding_program, dtype: int64

In [5]:
df = bc>>filter(_.funding_program.str.contains('5310'))


In [6]:
len(df>>count(_.organization_name))

147

In [7]:
df.sample()

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,project_closed_by,project_closed_date,project_closed_time
1408,2019,5310 Trad,CA-2020-244 | 0020000273-T,2019,On Lok Senior Health Services,BCG0001914,Purchase Replacement < 30 Ft Bus,111204,64AC19-01207,56960.0,71200.0,67209.89,-10249.89,0,Open,NaN,NaN,NaN


In [8]:
df.project_year.value_counts()

2019    604
2017    507
2018      4
Name: project_year, dtype: int64

In [9]:
from shared_utils import geography_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You m

In [10]:
org = geography_utils.aggregate_by_geography(
    df,
    group_cols=["organization_name"],
    nunique_cols=["grant_fiscal_year", "grant_number", "contract_number"],
    count_cols=["upin"],
    sum_cols=[
        "allocationamount",
        "encumbered_amount",
        "expendedamount",
        "activebalance",
        "closedoutbalance",
    ],
    mean_cols=[
        "allocationamount",
        "encumbered_amount",
        "expendedamount",
        "activebalance",
        "closedoutbalance",
    ],
    rename_cols=True,
)

In [11]:
org

,organization_name,activebalance_sum,allocationamount_sum,closedoutbalance_sum,encumbered_amount_sum,expendedamount_sum,activebalance_mean,allocationamount_mean,closedoutbalance_mean,encumbered_amount_mean,expendedamount_mean,upin_count,contract_number_nunique,grant_fiscal_year_nunique,grant_number_nunique
0,Amador Transit,-268853.00,469995.0,0,738848.0,738848.00,-67213.25000,117498.750,0,184712.000,184712.00000,4,3,2,3
1,"Angel View, Inc.",25729.61,554995.0,0,774990.0,529265.39,3216.20125,69374.375,0,96873.750,66158.17375,8,4,2,4
2,ARC Imperial Valley,-44628.20,549178.0,0,865145.0,593806.20,-5578.52500,68647.250,0,108143.125,74225.77500,8,3,2,3
3,Area 1 Agency on Aging,0.00,146534.0,0,146534.0,146534.00,0.00000,146534.000,0,146534.000,146534.00000,1,1,1,1
4,"Asian Community Center of Sacramento Valley, I...",541809.50,1425169.0,0,2320371.0,883359.50,27090.47500,71258.450,0,116018.550,44167.97500,20,5,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Self-Help for the Elderly,242098.00,242098.0,0,484196.0,0.00,30262.25000,30262.250,0,60524.500,0.00000,8,1,1,1
143,The Respite Inn,56000.00,56000.0,0,112000.0,0.00,28000.00000,28000.000,0,56000.000,0.00000,2,1,1,1
144,"Ukiah Senior Center, Inc.",56000.00,56000.0,0,112000.0,0.00,28000.00000,28000.000,0,56000.000,0.00000,2,1,1,1
145,United Cerebral Palsy Association Inc of Stani...,-239007.28,284800.0,0,569600.0,523807.28,-29875.91000,35600.000,0,71200.000,65475.91000,8,1,1,1


In [83]:
org_name= org>>select(_.organization_name)

In [84]:
org_name['organization_name'] = org_name['organization_name'].str.replace(',','').str.replace('.','').str.replace('/',' ')

/tmp/ipykernel_944/3813088631.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [85]:
org_name

,organization_name
0,Amador Transit
1,Angel View Inc
2,ARC Imperial Valley
3,Area 1 Agency on Aging
4,Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services
...,...
142,Self-Help for the Elderly
143,The Respite Inn
144,Ukiah Senior Center Inc
145,United Cerebral Palsy Association Inc of Stanislaus County


In [75]:
#pd.set_option("display.max_colwidth", 100)

In [74]:
org_name>>filter(_.organization_name.str.contains('/'))

,organization_name
12,Community Bridges/Liftline
60,Butte County Association of Governments/ Butte Regional Transit


In [86]:
org_name>>filter(_.organization_name.str.contains('Community Bridges'))

,organization_name
12,Community Bridges Liftline


In [56]:
# import re

In [139]:
# ## returns string
# def clean_string(row):
#     text = text = re.sub(r"[^A-z\s/]", "", row)
#      return text

In [142]:
# cleaned_org_name = [clean_string(row)for row in org_name.organization_name] 

## Read in Airtable Organization Information

In [113]:
itp= to_snakecase(pd.read_csv("gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/organizations_All_Organizations.csv"))


In [114]:
itp.sample()

,name,organization_type,roles,record_creation_time,ntp_id,itp_id,opm_id_drmt,dotid,brand,alias,details,website,parent_organization,administrating_organization,mobility_services_managed,missing_static,funding_sources_for_managed_transportation,mobility_services_operated,gtfs_datasets_produced,service_type__from_mobility_services_managed_,currently_operating__from_mobility_services_managed_,currently_operating__from_mobility_services_operated_,service_type__from_mobility_services_operated_,headquarters_place,funding_programs,total_voms__ntd_,service_area_sq_miles__ntd_,service_area_population__ntd_,caltrans_district,mpo_rtpa,planning_authority,tracking_category,reporting_category,assist_category,eligibility_programs,gtfs_datasets,gtfs_dataset__from_mobility_services_managed_,fare_systems,gtfs_schedule_status,fares_v2_status,flex_status,services_needing_alerts,services_needing_tripupdates_or_vehiclepositions,#_of_fixed_route_services,#_services_w__complete_rt_status,#_fixed_route_services_w__static_gtfs,complete_static_gtfs_coverage__1=yes_,complete_rt_coverage,>=1_gtfs_feed_for_any_service__1=yes_,>=_1_complete_rt_set__1=yes_,service_availability_category__from_mobility_services_managed_,#_fixed_route_or_deviated_fixed_route_services,#_fixed_route_or_deviated_fixed_route_service_w__static_gtfs,#_services_with_missing_static_feed_for_fixed_route_or_deviated_fixed_route,county_geography,county_geography_3,gtfs_static_status,gtfs_realtime_status,service_availability,service___component
631,Rio Hondo College,University - Public,NaN,10/5/2021 4:47pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rio Hondo College TRAM,Rio Hondo College TRAM,public,Rio Hondo College TRAM,NaN,fixed-route,1 checked out of 1,1 checked out of 1,fixed-route,Whittier,NaN,NaN,NaN,NaN,07 - Los Angeles,NaN,NaN,Active,Other Public Transit,Supportive,NaN,NaN,NaN,NaN,needed,NaN,NaN,Rio Hondo College TRAM,Rio Hondo College TRAM,1,0,0,0,0,0,0,Public\n,1,0,1,NaN,NaN,Static Incomplete,RT Incomplete,Public\n,NaN


In [115]:
itp = itp >> select(
    _.name,
    _.organization_type,
    _.roles,
    _.ntp_id,
    _.itp_id,
    _.alias,
    _.opm_id_drmt,
    _.mobility_services_operated,
    _.mpo_rtpa,
)

In [116]:
itp['name'] = itp['name'].str.replace(',','').str.replace('.','').str.replace('/',' ')

/tmp/ipykernel_944/3661866853.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [120]:
itp>>select(_.name)

,name
0,A-Paratransit
1,ABC Shuttle
2,Able Inc
3,Abrazar Inc
4,Access Services
...,...
823,Yosemite Area Regional Transportation System
824,Yuba-Sutter Transit Authority
825,Yuma County Intergovernmental Public Transportation Authority
826,Yurok Tribe


In [118]:
itp.rename(columns = {'ntp_id':'ntd_id'}, inplace=True)

## Read in NTD Agency Info

In [126]:
ntd = to_snakecase(pd.read_excel("gs://calitp-analytics-data/data-analyses/5311 /2020_Agency_Information.xlsx"))

In [127]:
ntd = ntd>>filter(_.state=='CA')

In [128]:
ntd = ntd >> select(
    _.ntd_id,
    _.agency_name,
    _.doing_business_as,
    _.reporter_acronym,
    _.doing_business_as,
    _.organization_type,
    _.reporter_type,
    _.reported_by_ntd_id,
    _.city,
)

In [129]:
ntd

,ntd_id,agency_name,doing_business_as,reporter_acronym,organization_type,reporter_type,reported_by_ntd_id,city
884,90003,San Francisco Bay Area Rapid Transit District,NaN,BART,Independent Public Agency or Authority of Transit Service,Full Reporter,NaN,Oakland
885,90004,Golden Empire Transit District,NaN,GET,Independent Public Agency or Authority of Transit Service,Full Reporter,NaN,Bakersfield
887,90006,Santa Cruz Metropolitan Transit District,NaN,SCMTD,Independent Public Agency or Authority of Transit Service,Full Reporter,NaN,Santa Cruz
888,90007,City of Modesto,Modesto Area Express,MAX,"City, County or Local Government Unit or Department of Transportation",Full Reporter,NaN,Modesto
889,90008,City of Santa Monica,Big Blue Bus,NaN,"City, County or Local Government Unit or Department of Transportation",Full Reporter,NaN,Santa Monica
...,...,...,...,...,...,...,...,...
2869,A0008,Kern Regional Transit,NaN,KT,"City, County or Local Government Unit or Department of Transportation",Group Plan Sponsor,NaN,Bakersfield
2870,A0009,Humboldt Transit Authority,NaN,HTA,Independent Public Agency or Authority of Transit Service,Group Plan Sponsor,NaN,Eureka
2952,A0013,Metropolitan Transportation Commission,NaN,MTC,"MPO, COG or Other Planning Agency",Group Plan Sponsor,NaN,San Francisco
2955,A0016,Redding Area Bus Authority,NaN,RABA,Independent Public Agency or Authority of Transit Service,Group Plan Sponsor,NaN,Redding


In [132]:
ntd['agency_name'] = ntd['agency_name'].str.replace(',','').str.replace('.','').str.replace('/',' ')

/tmp/ipykernel_944/652493469.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


## Merge NTD & ITP info

#### On Name

In [134]:
m = pd.merge(ntd, itp, left_on="agency_name", right_on="name", how="outer", indicator=True)

In [135]:
m._merge.value_counts()

right_only    649
both          181
left_only      60
Name: _merge, dtype: int64

In [138]:
m.sample(5)

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,name,organization_type_y,roles,ntd_id_y,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge
796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Southern California Association of Governments,Joint Powers Agency,"Metropolitan Planning Organization,Regional Transportation Planning Agency",NaN,NaN,NaN,NaN,NaN,NaN,right_only
519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Demaris Express Inc,Company,NaN,NaN,NaN,NaN,NaN,Cross Border Express,NaN,right_only
510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Costa Mesa Senior Center,Non-Profit Organization,NaN,NaN,80.0,NaN,NaN,Costa Mesa Senior Taxi,Southern California Association of Governments,right_only
201,9R02-91058,City of Ojai,NaN,NaN,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Ojai,City of Ojai,City/Town,NaN,9R02-91058,231.0,NaN,756009.0,Ojai Trolley,Southern California Association of Governments,both
535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,El Paseo Limo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only


In [140]:
nomatch1 = m >> filter(_._merge == "left_only")>>select(_.ntd_id_x,
                                                        _.agency_name,
                                                       _.doing_business_as, 
                                                        _.reporter_acronym,
                                                        _.organization_type_x,
                                                       _.reporter_type,
                                                       _.reported_by_ntd_id,
                                                       _.city)

In [141]:
nomatch2= m >> filter(_._merge == "right_only")>>select(_.name,
                                                        _.organization_type_y,
                                                        _.roles,
                                                        _.ntd_id_y,
                                                        _.itp_id, 
                                                        _.alias,
                                                        _.opm_id_drmt,
                                                        _.mobility_services_operated,
                                                        _.mpo_rtpa)

In [142]:
nomatch1.sample()

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city
180,9R02-91008,Modoc Transportation Agency,NaN,MTA,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Alturas


In [143]:
nomatch_merge = pd.merge(nomatch1, nomatch2, left_on="doing_business_as", right_on="name", how="outer", indicator=True)

In [144]:
to_add1 = nomatch_merge>>filter(_._merge=='both')

In [145]:
to_add1

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,name,organization_type_y,roles,ntd_id_y,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge
51,90236,Stanislaus County,Stanislaus Regional Transit,StaRT,"City, County or Local Government Unit or Department of Transportation",Reduced Reporter,NaN,Modesto,Stanislaus Regional Transit,Independent Agency,NaN,90236,325.0,NaN,1050003.0,"Stanislaus Regional Transit,StaRT Commuter,Medivan,Stanislaus Dial-A-Ride,Stanislaus Shuttles",Stanislaus Council of Governments,both


In [150]:
nomatch_merge = pd.merge(nomatch1, nomatch2, left_on="ntd_id_x", right_on="ntd_id_y", how="outer", indicator=True)

In [151]:
# no matches
to_add2 = nomatch_merge>>filter(_._merge=='both')
to_add2

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,name,organization_type_y,roles,ntd_id_y,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge
43,9R02-91048,County of Siskiyou,Siskiyou County Transit,STAGE,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Yreka,Siskiyou County,County,NaN,9R02-91048,83.0,STAGE,247002.0,Siskiyou Transit and General Express,NaN,both
44,9R02-91057,Tuolumne County Transit,NaN,TCT,Independent Public Agency or Authority of Transit Service,Rural Reporter,9R02,Sonora,Tuolumne County Transit Agency,Independent Agency,NaN,9R02-91057,482.0,NaN,1055004.0,"Tuolumne County Transit,Dodge Ridge Ski Bus,Tuolumne County Dial-Your-Ride",NaN,both
45,9R02-91059,Kern Regional Transit,NaN,KT,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Bakersfield,Kern County,County,NaN,9R02-91059,146.0,NaN,615006.0,"Kern Transit,Mojave Dial-A-Ride,Rosamonde Dial-A-Ride,Tehachapi Dial-A-Ride,Frazier Park Dial-A-...",Kern Council of Governments,both
48,9R02-91088,Glenn Transit Service,NaN,GTS,Independent Public Agency or Authority of Transit Service,Rural Reporter,9R02,Willows,Glenn County,County,NaN,9R02-91088,122.0,NaN,311002.0,NaN,NaN,both
49,9R02-91093,City of Eureka,Eureka Transit Service,ETS,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Eureka,City of Eureka,City/Town,NaN,9R02-91093,108.0,NaN,112003.0,Eureka Transit Service,NaN,both
50,9R02-91095,Nevada County Transit Services,NaN,NaN,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Nevada City,Nevada County,County,NaN,9R02-91095,221.0,Gold Country Stage,329002.0,"Nevada County Connects,Nevada County Now",NaN,both
52,9R02-99438,County of Shasta Department of Public Works,NaN,NaN,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Redding,Shasta County,County,NaN,9R02-99438,NaN,NaN,NaN,NaN,NaN,both


In [152]:
nomatch_merge = pd.merge(nomatch1, nomatch2, left_on="reporter_acronym", right_on="alias", how="outer", indicator=True)

In [153]:
to_add3 = nomatch_merge>>filter(_._merge=='both')
to_add4 = (to_add3>>filter(_.alias.notnull()))
to_add4

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,name,organization_type_y,roles,ntd_id_y,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge
0,90007,City of Modesto,Modesto Area Express,MAX,"City, County or Local Government Unit or Department of Transportation",Full Reporter,NaN,Modesto,City of Modesto,City/Town,NaN,90007,203.0,MAX,1050002.0,"Modesto Area Express,eTrans Dial-A-Ride,Modesto Area Dial-A-Ride",Stanislaus Council of Governments,both
12879,90092,City of Fairfield California,Fairfield and Suisun Transit,FAST,"City, County or Local Government Unit or Department of Transportation",Full Reporter,NaN,Fairfield,City of Fairfield,City/Town,NaN,90092,110.0,FAST,448002.0,"Fairfield and Suisun Transit,DART Paratransit,Solano Express",Metropolitan Transportation Commission,both
12880,90144,Livermore Amador Valley Transit Authority,NaN,LAVTA,Independent Public Agency or Authority of Transit Service,Full Reporter,NaN,Livermore,Livermore Amador\n Valley Transit Authority,Independent Agency,NaN,90144,167.0,LAVTA,401003.0,"Tri-Valley Wheels,Tri-Valley Wheels Paratransit,Wheels School",Metropolitan Transportation Commission,both
12883,90182,Altamont Corridor Express,NaN,ACE,Independent Public Agency or Authority of Transit Service,Full Reporter,NaN,Stockton,San Joaquin Regional Rail Commission,Independent Agency,NaN,90182,10.0,ACE,1039003.0,Altamont Corridor Express,San Joaquin Council of Governments,both
12885,90206,San Luis Obispo Regional Transit Authority,NaN,SLORTA,Independent Public Agency or Authority of Transit Service,Full Reporter,NaN,San Luis Obispo,San Luis Obispo Regional Transit Authority,Independent Agency,NaN,90206,289.0,SLORTA,540005.0,"South County Area Transit,San Luis Obispo Regional Transit Authority,Paso Express,Avila Beach Tr...",San Luis Obispo Council of Governments,both
12886,A0022,San Luis Obispo Regional Transit Authority,NaN,SLORTA,Independent Public Agency or Authority of Transit Service,Group Plan Sponsor,NaN,San Luis Obispo,San Luis Obispo Regional Transit Authority,Independent Agency,NaN,90206,289.0,SLORTA,540005.0,"South County Area Transit,San Luis Obispo Regional Transit Authority,Paso Express,Avila Beach Tr...",San Luis Obispo Council of Governments,both
12902,9R02-91048,County of Siskiyou,Siskiyou County Transit,STAGE,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Yreka,Siskiyou County,County,NaN,9R02-91048,83.0,STAGE,247002.0,Siskiyou Transit and General Express,NaN,both


In [154]:
## Subsetting original merge

In [155]:
match = m>>filter(_._merge=='both')

In [156]:
match.sample()

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,name,organization_type_y,roles,ntd_id_y,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge
209,9R02-91078,City of Escalon,eTrans,NaN,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Escalon,City of Escalon,City/Town,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [157]:
matches = pd.concat([match, to_add1], ignore_index=True, sort=False)

In [158]:
matches = pd.concat([matches, to_add2], ignore_index=True, sort=False)

In [159]:
matches = pd.concat([matches, to_add4], ignore_index=True, sort=False)

In [164]:
matches.sample()


,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,name,organization_type_y,roles,ntd_id_y,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge
122,99256,Susanville Indian Rancheria,NaN,SIR,Tribe,Reduced Reporter,NaN,Susanville,Susanville Indian Rancheria,Tribe,NaN,99256,329.0,NaN,NaN,NaN,NaN,both


## Merge Matches with 5310

In [162]:
#first fill in the missing information

In [166]:
matches>>filter(_.reporter_acronym.isnull())>>filter(_.alias.notnull())

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,name,organization_type_y,roles,ntd_id_y,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge
8,90014,Alameda-Contra Costa Transit District,AC Transit,NaN,Independent Public Agency or Authority of Transit Service,Full Reporter,NaN,Oakland,Alameda-Contra Costa Transit District,Independent Agency,NaN,90014,4.0,AC Transit,401001.0,"AC Transit,AC Transit Transbay,AC Transit Tempo",Metropolitan Transportation Commission,both
12,90019,Sacramento Regional Transit District,Sacramento RT,NaN,Independent Public Agency or Authority of Transit Service,Full Reporter,NaN,Sacramento,Sacramento Regional Transit District,Independent Agency,NaN,90019,273.0,"SacRT,Sacramento RT",334004.0,"SacRT Light Rail,e-van,SacRT GO,Elk Grove Transit Services,Folsom Stage Line,Causeway Connection...",Sacramento Area Council of Governments,both
22,90042,City of Gardena,GTrans,NaN,"City, County or Local Government Unit or Department of Transportation",Full Reporter,NaN,Gardena,City of Gardena,City/Town,NaN,NaN,118.0,GTrans,719008.0,"GTrans,GTrans Special Transit",Southern California Association of Governments,both
24,90044,City of Arcadia,Arcadia Transit,NaN,"City, County or Local Government Unit or Department of Transportation",Reduced Reporter,NaN,Arcadia,City of Arcadia,City/Town,NaN,90044,17.0,Arcadia Transit,719003.0,"Arcadia Transit,Arcadia Transit Dial-A-Ride",Southern California Association of Governments,both
48,90156,City of San Luis Obispo,SLO Transit,NaN,"City, County or Local Government Unit or Department of Transportation",Full Reporter,NaN,San Luis Obispo,City of San Luis Obispo,City/Town,NaN,90156,287.0,SLO Trans,540004.0,SLO Transit,San Luis Obispo Council of Governments,both
107,90287,Palos Verdes Peninsula Transit Authority,NaN,NaN,Independent Public Agency or Authority of Transit Service,Reduced Reporter,NaN,Palos Verdes Peninsula,Palos Verdes Peninsula Transit Authority,Independent Agency,NaN,90287,239.0,Desert Roadrunner,719029.0,Palos Verdes Peninsula Transit Authority,Southern California Association of Governments,both
131,99423,City of Glendale,NaN,NaN,"City, County or Local Government Unit or Department of Transportation",Full Reporter,NaN,Glendale,City of Glendale,City/Town,NaN,99423,120.0,Beeline,NaN,"Glendale Beeline,Glendale Dial-A-Ride",Southern California Association of Governments,both
171,9R02-91112,Colusa County Transit Agency,NaN,NaN,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Colusa,Colusa County Transit Agency,Independent Agency,NaN,9R02-91112,74.0,CCTA,306002.0,Colusa County Transit,NaN,both
175,9R02-99454,Palo Verde Valley Transit Agency,NaN,NaN,Independent Public Agency or Authority of Transit Service,Rural Reporter,9R02,Blythe,Palo Verde Valley Transit Agency,Independent Agency,NaN,NaN,238.0,"PVVTA,Desert Roadrunner",833005.0,Desert Roadrunner,Southern California Association of Governments,both
187,9R02-91095,Nevada County Transit Services,NaN,NaN,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Nevada City,Nevada County,County,NaN,9R02-91095,221.0,Gold Country Stage,329002.0,"Nevada County Connects,Nevada County Now",NaN,both


In [167]:
matches.reporter_acronym.fillna(matches['alias'], inplace=True)

In [179]:
compare_col = np.where(matches["reporter_acronym"] == matches["alias"], True, False)
matches["compare_acronym"] = compare_col
matches.compare_acronym.value_counts()

False    166
True      30
Name: compare_acronym, dtype: int64

In [182]:
matches>>filter(_.compare_acronym==False)>>filter(_.reporter_acronym.notnull())>>filter(_.alias.notnull())>>select(_.agency_name, _.name, _.reporter_acronym, _.alias)

,agency_name,name,reporter_acronym,alias
6,San Joaquin Regional Transit District,San Joaquin Regional Transit District,RTD,SJRTD
7,Santa Clara Valley Transportation Authority,Santa Clara Valley Transportation Authority,VTA,SCVTA
9,City and County of San Francisco,City and County of San Francisco,SFMTA,"Muni,SF Muni,SFMTA"
13,Santa Barbara Metropolitan Transit District,Santa Barbara Metropolitan Transit District,SBMTD,"""Santa Barbara MTD\trecswCrw6a6htmXJ4\t0\t\tMobility service provider, GTFS data provider\t\t\tS..."
15,San Diego Metropolitan Transit System,San Diego Metropolitan Transit System,MTS,SDMTD
16,City of Fresno,City of Fresno,FAX,"Fresno Area Express,FAX"
35,Metropolitan Transportation Commission,Metropolitan Transportation Commission,MTC,"MTC,Bay Area Metro,Bay Area 511"
36,Metropolitan Transportation Commission,Metropolitan Transportation Commission,MTC,"MTC,Bay Area Metro,Bay Area 511"
40,Peninsula Corridor Joint Powers Board,Peninsula Corridor Joint Powers Board,PCJPB,"Caltrain,PCJPB"
43,City of Los Angeles,City of Los Angeles,LADOT,"LA DOT,Los Angeles DOT"


In [ ]:
#matches.drop(columns={alias}

In [171]:
matches>>filter(_.organization_type_x.isnull())>>filter(_.organization_type_y.notnull())

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,name,organization_type_y,roles,ntd_id_y,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge


In [172]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   ntd_id_x                    196 non-null    object  
 1   agency_name                 196 non-null    object  
 2   doing_business_as           60 non-null     object  
 3   reporter_acronym            121 non-null    object  
 4   organization_type_x         195 non-null    object  
 5   reporter_type               196 non-null    object  
 6   reported_by_ntd_id          53 non-null     object  
 7   city                        196 non-null    object  
 8   name                        196 non-null    object  
 9   organization_type_y         179 non-null    object  
 10  roles                       10 non-null     object  
 11  ntd_id_y                    177 non-null    object  
 12  itp_id                      187 non-null    float64 
 13  alias               

In [173]:
matches>>filter(_.roles.notnull())

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,name,organization_type_y,roles,ntd_id_y,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge
35,90094,Metropolitan Transportation Commission,NaN,MTC,"MPO, COG or Other Planning Agency",Full Reporter,NaN,San Francisco,Metropolitan Transportation Commission,MPO/RTPA,Metropolitan Planning Organization,NaN,200.0,"MTC,Bay Area Metro,Bay Area 511",NaN,NaN,NaN,both
36,A0013,Metropolitan Transportation Commission,NaN,MTC,"MPO, COG or Other Planning Agency",Group Plan Sponsor,NaN,San Francisco,Metropolitan Transportation Commission,MPO/RTPA,Metropolitan Planning Organization,NaN,200.0,"MTC,Bay Area Metro,Bay Area 511",NaN,NaN,NaN,both
37,90095,San Diego Association of Governments,NaN,SANDAG,"MPO, COG or Other Planning Agency",Full Reporter,NaN,San Diego,San Diego Association of Governments,NaN,"Metropolitan Planning Organization,Regional Transportation Planning Agency,Census Affiliate Data...",90095,276.0,NaN,NaN,NaN,NaN,both
52,90164,Ventura County Transportation Commission,NaN,VCTC,Independent Public Agency or Authority of Transit Service,Full Reporter,NaN,Camarillo,Ventura County Transportation Commission,NaN,Regional Transportation Planning Agency,90164,380.0,NaN,756001.0,"Valley Express,VCTC Intercity,Valley Express Dial-A-Ride",Southern California Association of Governments,both
66,90208,Butte County Association of Governments,NaN,BCAG,"MPO, COG or Other Planning Agency",Full Reporter,NaN,Chico,Butte County Association of Governments,Joint Powers Agency,"Regional Transportation Planning Agency,Metropolitan Planning Organization,Census Affiliate Data...",90208,47.0,NaN,304001.0,"B-Line,B-Line Paratransit",Butte County Association of Governments,both
117,90297,San Luis Obispo Council of Governments,NaN,SLOCOG,"MPO, COG or Other Planning Agency",Reduced Reporter,NaN,San Luis Obispo,San Luis Obispo Council of Governments,Council of Governments,"Regional Transportation Planning Agency,Metropolitan Planning Organization,Census Affiliate Data...",90297,288.0,SLOCOG,NaN,Paso Robles Dial-A-Ride,NaN,both
121,90303,Santa Barbara County Association of Governments,NaN,SBCAG,"MPO, COG or Other Planning Agency",Reduced Reporter,NaN,Santa Barbara,Santa Barbara County Association of Governments,NaN,"Metropolitan Planning Organization,Regional Transportation Planning Agency,Local Transportation ...",NaN,379.0,SBCAG,542006.0,Clean Air Express,Santa Barbara County Association of Governments,both
150,9R02-91038,Sierra County Transportation Commission,NaN,SCTC,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Downieville,Sierra County Transportation Commission,NaN,Regional Transportation Planning Agency,9R02-91038,304.0,NaN,NaN,NaN,NaN,both
172,9R02-91116,Alpine County Local Transportation Commission,NaN,NaN,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Markleeville,Alpine County Local Transportation Commission,NaN,Regional Transportation Planning Agency,NaN,NaN,NaN,NaN,NaN,NaN,both
173,9R02-91119,Plumas County Transportation Commission,NaN,PCTC,"City, County or Local Government Unit or Department of Transportation",Rural Reporter,9R02,Quincy,Plumas County Transportation Commission,NaN,Regional Transportation Planning Agency,NaN,NaN,NaN,NaN,NaN,NaN,both


In [176]:
def recategorize(df):   
    if (df.roles.str.contains('Metropolitan Planning Organization')):
        return 'MPO/RTPA'
    elif (df.roles.str.contains('Regional Transportation Planning Agency')) :
        return 'MPO/RTPA'
    elif (dfroles.str.contains('Regional Transportation Planning Agency,Metropolitan Planning Organization')):
        return 'MPO/RTPA'
    else: 
        return NaN

In [177]:
matches['organization_type_z'] = matches.apply(recategorize, axis = 1)

AttributeError: 'float' object has no attribute 'str'

In [163]:
#agency_info = pd.merge(matches, nomatch2, left_on="reporter_acronym", right_on="alias", how="outer", indicator=True)

## Merging 5310 Org names to CW

In [74]:
org_name.sample()

,organization_name
73,El Dorado County Transit Authority


In [73]:
matches.sample()

,ntd_id,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,name,organization_type_y,roles,ntp_id,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge
48,90156,City of San Luis Obispo,SLO Transit,NaN,"City, County or Local Government Unit or Depar...",Full Reporter,NaN,San Luis Obispo,City of San Luis Obispo,City/Town,NaN,90156,287.0,SLO Trans,540004.0,SLO Transit,San Luis Obispo Council of Governments,both


In [81]:
(pd.merge(org_name, matches, left_on="organization_name", right_on="agency_name", how="outer", indicator="in_5310")).in_5310.value_counts()

right_only    169
left_only     122
both           27
Name: in_5310, dtype: int64

In [144]:
itp

,organization_name,organization_type,roles,ntp_id,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa
0,A-Paratransit,NaN,NaN,NaN,NaN,NaN,NaN,East Bay Paratransit,NaN
1,ABC Shuttle,Company,NaN,NaN,NaN,NaN,NaN,ABC Airport Shuttle,NaN
2,Able Inc.,Non-Profit Organization,NaN,NaN,NaN,NaN,NaN,Able Inc.,NaN
3,Abrazar Inc.,Non-Profit Organization,NaN,NaN,NaN,NaN,NaN,Abrazar,NaN
4,Access Services,NaN,NaN,90157,1.0,NaN,NaN,LA Access Services,Southern California Association of Governments
...,...,...,...,...,...,...,...,...,...
823,Yosemite Area Regional Transportation System,Independent Agency,NaN,9R02-91070,374.0,YARTS,1024004.0,Yosemite Area Regional Transportation System,Merced County Association of Governments
824,Yuba-Sutter Transit Authority,Independent Agency,NaN,90061,376.0,NaN,358001.0,"Yuba-Sutter Transit,Yuba-Sutter Rural Transit,...",Sacramento Area Council of Governments
825,Yuma County Intergovernmental Public Transport...,Independent Agency,NaN,90233,386.0,NaN,1113004.0,Yuma County Area Transit,Southern California Association of Governments
826,Yurok Tribe,Tribe,NaN,99262,377.0,NaN,NaN,Yurok Tribe Transit Service,NaN


In [145]:
itp_name = itp>>select(_.organization_name, _.ntp_id)

In [146]:
#itp = itp.rename(columns={'name':'organization_name'})

In [167]:
t1 = pd.merge(org_name, itp, on="organization_name", how="outer", indicator=True)

In [168]:
t1._merge.value_counts()

right_only    769
left_only      88
both           59
Name: _merge, dtype: int64

In [169]:
needmatch_5310_t1 = t1>>filter(_._merge=='left_only')

In [170]:
t1>>filter(_._merge=="both")

,organization_name,organization_type,roles,ntp_id,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge
2,ARC Imperial Valley,Non-Profit Organization,NaN,NaN,398.0,NaN,NaN,Arc Imperial Valley Transportation,NaN,both
3,Area 1 Agency on Aging,NaN,NaN,NaN,399.0,NaN,NaN,NaN,NaN,both
6,Catholic Charities of the Diocese of Stockton,Non-Profit Organization,NaN,NaN,408.0,NaN,NaN,Catholic Charities Stockton Senior Transportation,NaN,both
9,City of Roseville,City/Town,NaN,90168,271.0,NaN,331005.0,"Roseville Transit,Roseville Dial-A-Ride",Sacramento Area Council of Governments,both
21,Eastern Sierra Transit Authority,Independent Agency,NaN,9R02-91062,99.0,ESTA,926002.0,Eastern Sierra Transit Authority Community Rou...,NaN,both
22,ElderHelp of San Diego,Non-Profit Organization,NaN,NaN,427.0,NaN,NaN,Seniors A Go Go,NaN,both
25,Friends of Children with Special Needs,Non-Profit Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
27,Independent Living Partnership,NaN,NaN,NaN,433.0,NaN,NaN,NaN,NaN,both
28,Lake Transit Authority,Independent Agency,NaN,9R02-91053,159.0,NaN,117002.0,"Lake Transit,Clearlake/Lower Lake Dial-A-Ride,...",NaN,both
29,LIFE ElderCare,Non-Profit Organization,NaN,NaN,437.0,NaN,NaN,Life ElderCare,NaN,both


In [171]:
ntd_names = ntd>>select(_.organization_name, _.doing_business_as, _.ntd_id)

In [154]:
#ntd = ntd.rename(columns={'agency_name':'organization_name'})

In [172]:
t2 = pd.merge(org_name, ntd, on="organization_name", how="outer", indicator=True)

In [173]:
t2._merge.value_counts()

right_only    213
left_only     121
both           28
Name: _merge, dtype: int64

In [174]:
needmatch_5310_t2 =  t2>>filter(_._merge=='left_only')

In [175]:
t2>>filter(_._merge=='both')

,organization_name,ntd_id,doing_business_as,reporter_acronym,organization_type,reporter_type,reported_by_ntd_id,city,_merge
9,City of Roseville,90168,Roseville Transit,NaN,"City, County or Local Government Unit or Depar...",Reduced Reporter,NaN,Roseville,both
21,Eastern Sierra Transit Authority,9R02-91062,NaN,ESTA,Independent Public Agency or Authority of Tran...,Rural Reporter,9R02,Bishop,both
28,Lake Transit Authority,9R02-91053,NaN,LTA,Independent Public Agency or Authority of Tran...,Rural Reporter,9R02,Lower Lake,both
31,Marin County Transit District,90234,NaN,NaN,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,San Rafael,both
34,Morongo Basin Transit Authority,9R02-91090,NaN,MBTA,Independent Public Agency or Authority of Tran...,Rural Reporter,9R02,Joshua Tree,both
41,Riverside Transit Agency,90031,NaN,RTA,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,Riverside,both
43,San Mateo County Transit District,90009,SamTrans,NaN,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,San Carlos,both
44,Santa Clara Valley Transportation Authority,90013,NaN,VTA,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,San Jose,both
49,Town of Truckee,9R02-91101,NaN,TOT,"City, County or Local Government Unit or Depar...",Rural Reporter,9R02,Truckee,both
54,Yolo County Transportation District,90090,Yolobus,YCTD,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,Woodland,both


In [176]:
t1.sample()

,organization_name,organization_type,roles,ntp_id,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge
0,Amador Transit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [177]:
t1 = t1>>filter(_._merge=='both')
t1 = t1.drop(columns={'_merge'})

In [178]:
t2 = t2>>filter(_._merge=='both')
t2 = t2.drop(columns={'_merge'})

## 5310 Matches

In [179]:
_5310info = pd.concat([t1, t2], ignore_index=True, sort=False)

In [180]:
_5310info

,organization_name,organization_type,roles,ntp_id,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,ntd_id,doing_business_as,reporter_acronym,reporter_type,reported_by_ntd_id,city
0,ARC Imperial Valley,Non-Profit Organization,NaN,NaN,398.0,NaN,NaN,Arc Imperial Valley Transportation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Area 1 Agency on Aging,NaN,NaN,NaN,399.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Catholic Charities of the Diocese of Stockton,Non-Profit Organization,NaN,NaN,408.0,NaN,NaN,Catholic Charities Stockton Senior Transportation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,City of Roseville,City/Town,NaN,90168,271.0,NaN,331005.0,"Roseville Transit,Roseville Dial-A-Ride",Sacramento Area Council of Governments,NaN,NaN,NaN,NaN,NaN,NaN
4,Eastern Sierra Transit Authority,Independent Agency,NaN,9R02-91062,99.0,ESTA,926002.0,Eastern Sierra Transit Authority Community Rou...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,San Luis Obispo Council of Governments,"MPO, COG or Other Planning Agency",NaN,NaN,NaN,NaN,NaN,NaN,NaN,90297,NaN,SLOCOG,Reduced Reporter,NaN,San Luis Obispo
83,Victor Valley Transit Authority,Independent Public Agency or Authority of Tran...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90148,NaN,VVTA,Full Reporter,NaN,Hesperia
84,City of Rio Vista,"City, County or Local Government Unit or Depar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9R02-91014,Rio Vista Delta Breeze,NaN,Rural Reporter,9R02,Rio Vista
85,Colusa County Transit Agency,"City, County or Local Government Unit or Depar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9R02-91112,NaN,NaN,Rural Reporter,9R02,Colusa


In [201]:
len(_5310info>>count(_.organization_name)>>arrange(-_.n)>>filter(_.n>1))

25

In [198]:
_5310info = _5310info.drop_duplicates()

In [202]:
#still have duplicates
len(_5310info>>count(_.organization_name)>>arrange(-_.n)>>filter(_.n>1))

25

In [208]:
_5310info>>filter(_.organization_name=='Humboldt Transit Authority')

,organization_name,organization_type,roles,ntp_id,itp_id,alias,opm_id_drmt,mobility_services_operated,mpo_rtpa,ntd_id,doing_business_as,reporter_acronym,reporter_type,reported_by_ntd_id,city
34,Humboldt Transit Authority,Independent Agency,NaN,9R02-91036,135.0,NaN,112005.0,"Humboldt Transit Authority Dial-A-Ride,Redwood...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,Humboldt Transit Authority,Independent Public Agency or Authority of Tran...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9R02-91036,NaN,HTA,Rural Reporter,9R02,Eureka
72,Humboldt Transit Authority,Independent Public Agency or Authority of Tran...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A0009,NaN,HTA,Group Plan Sponsor,NaN,Eureka


## 5310 still need matches

In [181]:
need = pd.concat([needmatch_5310_t1, needmatch_5310_t2], ignore_index=True, sort=False)

In [203]:
need = need>>select(_.organization_name)

In [204]:
need = need.drop_duplicates()

In [205]:
len(need)

122

In [206]:
need

,organization_name
0,Amador Transit
1,"Angel View, Inc."
2,"Asian Community Center of Sacramento Valley, I..."
3,Avenidas
4,Choice in Aging
...,...
187,Reach Out Morongo Basin
202,Porterville Sheltered Workshop
204,Self-Help for the Elderly
205,The Respite Inn
